# 各種ユーザーログ 
## 過去N問の正答率

## 過去N問の(うち同じパートでの)正答率

In [6]:
import cudf
import pandas as pd
print('cudf_version: ', cudf.__version__)
print('pd_version: ', pd.__version__)

cudf_version:  21.10.01
pd_version:  1.3.5


In [7]:
import pandas as pd
import numpy as np
import gc
from sklearn.metrics import roc_auc_score
from collections import defaultdict
from tqdm.notebook import tqdm
import lightgbm as lgb

## pandasの場合

In [33]:
%%time
validaten_flg = True

if validaten_flg:
    train_pickle = '../input/riiid-cross-validation-files/cv1_train.pickle'
    data = pd.read_pickle(train_pickle)
    data = data[:10000]

else:
    dtypes = {
    "row_id": "int64",
    "timestamp": "int64",
    "user_id": "int32",
    "content_id": "int16",
    "content_type_id": "boolean",
    "task_container_id": "int16",
    "user_answer": "int8",
    "answered_correctly": "int8",
    "prior_question_elapsed_time": "float32", 
    "prior_question_had_explanation": "boolean"
    }
    data = pd.read_csv("../input/riiid-test-answer-prediction/train.csv", dtype=dtypes)   

print("Train size:", data.shape)

Train size: (10000, 13)
CPU times: user 396 ms, sys: 2.52 s, total: 2.92 s
Wall time: 2.92 s


In [34]:
data.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,max_time_stamp,rand_time_stamp,viretual_time_stamp
0,32933156,0,705741139,128,0,0,0,1,NaN,<NA>,87425772049,0,0
1,32933157,20666,705741139,7860,0,1,0,1,16000.0,False,87425772049,0,20666
2,32933158,39172,705741139,7922,0,2,1,1,19000.0,False,87425772049,0,39172
3,32933159,58207,705741139,156,0,3,2,1,17000.0,False,87425772049,0,58207
4,32933160,75779,705741139,51,0,4,0,1,17000.0,False,87425772049,0,75779


In [99]:
# user_idごとにgroupby
df_base = data.query('user_answer != -1')[['timestamp', 'user_id', 'answered_correctly']]\
        .sort_values(['user_id','timestamp']).reset_index(drop=True)
df_base = df_base.groupby(["user_id", 'timestamp'])['answered_correctly'].mean().reset_index()

In [102]:
df_base

,user_id,timestamp,answered_correctly
0,4496897,0,1.0
1,4496897,15363,1.0
2,4496897,30198,1.0
3,4496897,52474,1.0
4,4496897,73937,1.0
...,...,...,...
7821,2144322103,223228,1.0
7822,2144322103,246293,1.0
7823,2144322103,370646,1.0
7824,2144322103,403714,1.0


In [103]:
grouped_data = df_base.groupby('user_id')

In [104]:
# user_idごとに過去の平均正答率を求める
# rolling関数を用いて、過去N問分の平均正答率を求める
N = 3
user_avg_correct = grouped_data["answered_correctly"].shift(1).rolling(N).mean()

In [105]:
user_avg_correct

0            NaN
1            NaN
2            NaN
3       1.000000
4       1.000000
          ...   
7821    0.666667
7822    1.000000
7823    1.000000
7824    1.000000
7825    1.000000
Name: answered_correctly, Length: 7826, dtype: float64

In [106]:
goal_df = pd.concat([df_base, user_avg_correct], axis='columns')

In [107]:
goal_df

,user_id,timestamp,answered_correctly,answered_correctly
0,4496897,0,1.0,NaN
1,4496897,15363,1.0,NaN
2,4496897,30198,1.0,NaN
3,4496897,52474,1.0,1.000000
4,4496897,73937,1.0,1.000000
...,...,...,...,...
7821,2144322103,223228,1.0,0.666667
7822,2144322103,246293,1.0,1.000000
7823,2144322103,370646,1.0,1.000000
7824,2144322103,403714,1.0,1.000000


In [108]:
goal_df.query('user_id == 1284718605').head(50)

,user_id,timestamp,answered_correctly,answered_correctly
4750,1284718605,0,1.000000,NaN
4751,1284718605,27618,0.000000,NaN
4752,1284718605,56266,0.000000,NaN
4753,1284718605,76712,1.000000,0.333333
4754,1284718605,153427,0.666667,0.333333
4755,1284718605,233059,1.000000,0.555556
4756,1284718605,321344,0.000000,0.888889
4757,1284718605,407661,0.666667,0.555556
4758,1284718605,427133,0.000000,0.555556
4759,1284718605,448337,1.000000,0.222222


In [9]:
import pickle

def pickle_dump(obj, path):
    with open(path, mode='wb') as f:
        pickle.dump(obj,f)

if validaten_flg:
    pickle_dump(gdf_user_answer.to_pandas(), '../input/my_validaten_datasets/user_answer_rate_cv1.pickle')
else:
    gdf_user_answer.to_csv('../input/user_answer_rate.csv', index=False)

In [10]:
# data_merge.to_pandas().to_csv('../input/all_train.csv', index=False)